<a href="https://colab.research.google.com/github/iamogbz/colab-research-gemini/blob/main/research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gemini: Connecting to Gemini

The Gemini API allows you to connect to Google's most powerful multi-modal model. This example configures your API key and sends an example message to the API and prints a response.

Before you start, visit https://aistudio.google.com/app/apikey to create an API key.

In [1]:
# @title Configure Gemini API key

import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = 'GOOGLE_API_KEY'  # @param {type: "string"}

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://aistudio.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}')
   raise e
except userdata.NotebookAccessError as e:
  print(f'You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.')
  raise e
except Exception as e:
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://aistudio.google.com/app/apikey")
  raise e

In [16]:
# @title Generate Steps For Researching Topic

import json


system_instructions = 'You are a helpful research assistant' # @param {type: "string"}
model = 'gemini-1.5-flash' # @param {type: "string"} ["gemini-1.0-pro", "gemini-1.5-pro", "gemini-1.5-flash"]
temperature = 0.5 # @param {type: "slider", min: 0, max: 2, step: 0.05}
# stop_sequence = '' # @param {type: "string"}

if model == 'gemini-1.0-pro' and system_instructions is not None:
  system_instructions = None
  print('\x1b[31m(WARNING: System instructions ignored, gemini-1.0-pro does not support system instructions)\x1b[0m')

if model == 'gemini-1.0-pro' and temperature > 1:
  temperature = 1
  print('\x1b[34m(INFO: Temperature set to 1, gemini-1.0-pro does not support temperature > 1)\x1b[0m')

if system_instructions == '':
  system_instructions = None

model = genai.GenerativeModel(model, system_instruction=system_instructions)
config = genai.GenerationConfig(temperature=temperature, stop_sequences=[])


def generate_research_steps(topic_title):
    prompt = f"""
    Generate a detailed list of steps a researcher should take to create a satisfactory response paper on the topic: "{topic_title}".
    The steps should be organized and numbered.  Include considerations for literature review, methodology, data collection (if applicable), analysis, and writing.
    Only output the steps in order and nothing else in the format of a plain JSON array string without markdown.
    Where each item is the step as plain text with no formatting or numbering.
    """

    try:
        response = model.generate_content(contents=[prompt], generation_config=config)
        return response.text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Usage
topic = "Automated Investigation and Research Assistants - Towards AI-powered Knowledge Discovery"  # @param {type: "string"}
research_steps_raw = generate_research_steps(topic)

if research_steps_raw:
    print("Research steps generated successfully:")
    print(research_steps_raw)
    research_steps = json.loads("\n".join(research_steps_raw.splitlines()[1:-1]))
else:
    print("Research steps generation failed.")

Research steps generated successfully:
```json
[
"Define the scope and focus of your response paper: Identify the specific aspects of \"Automated Investigation and Research Assistants - Towards AI-powered Knowledge Discovery\" that you will address. This could include specific AI techniques, applications, ethical considerations, or limitations.",
"Conduct a thorough literature review: Systematically search for and analyze relevant research papers, articles, books, and other scholarly resources related to AI-powered research assistants.  Focus on both the technical aspects and the broader implications.",
"Identify key themes and arguments in the existing literature: Summarize the main findings, debates, and perspectives on AI-powered knowledge discovery from your literature review.",
"Develop a clear thesis statement: Based on your literature review, formulate a concise and focused thesis statement that articulates the central argument or perspective of your response paper.",
"Structure